In [1]:
import pandas as pd
import numpy as np
import glob
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud


# Fungsi untuk membersihkan teks
def text_clean(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    sastrawi = StopWordRemoverFactory()
    stopworda = sastrawi.get_stop_words()
    clean_spcl = re.compile('[/(){}\[\]\|@,;]')
    clean_symbol = re.compile('[^0-9a-z #+_]')
    text = text.lower()
    text = clean_spcl.sub(' ', text)
    text = clean_symbol.sub('', text)
    text = stemmer.stem(text)
    text = ' '.join(word for word in text.split() if word not in stopworda)
    return text

def cleaning_judulabstract():
    # Pengecekan data baru pada 
    try:
        cleaned_data_skripsi = pd.read_csv('cleaned_data_skripsi.csv')
    except FileNotFoundError:
        cleaned_data_skripsi = pd.DataFrame()  # Jika file tidak ada, mulai dengan DataFrame kosong
    
    # Memuat data terbaru
    # data_skripsi_baru = pd.read_excel('data_skripsi_librarytrunojoyo.xlsx', sheet_name='skripsi', usecols='A:I')
    data_skripsi_baru = pd.read_excel('data_skripsi_librarytrunojoyo.xlsx', sheet_name='skripsi', usecols='A:I', engine='openpyxl')
    data_skripsi_baru = data_skripsi_baru[data_skripsi_baru['Abstrak'].notnull() & data_skripsi_baru['Judul Skripsi'].notnull()]


    #Cek apakah ada baris data baru di masukan kedalam excel
    if not cleaned_data_skripsi.empty:
        data_baru = data_skripsi_baru[~data_skripsi_baru['Judul Skripsi'].isin(cleaned_data_skripsi['Judul Skripsi'])]
    else:
        data_baru = data_skripsi_baru
    
    #Jika tidak ada rows data skripsi baru, maka melakukan cleaning
    if not data_baru.empty:
        print("Data baru ditemukan, melakukan proses NLP Cleaning")
        data_baru['cleaned_judul'] = data_baru['Judul Skripsi'].apply(text_clean)
        data_baru['cleaned_abstrak'] = data_baru['Abstrak'].apply(text_clean)

        updated_cleaned_data = pd.concat([cleaned_data_skripsi, data_baru], ignore_index=True)
        updated_cleaned_data.to_csv('cleaned_data_skripsi.csv', index=False)

        print("Proses Cleaning pada data telah selesai, data telah di simpan")
    else:
        print("Tidak ada rows data skripsi baru, cleaning tidak diperlukan")

# Fungsi untuk menampilkan data skripsi
def display_data_skripsi():
    data_skripsi = pd.read_csv('cleaned_data_skripsi.csv', sheet_name='skripsi', usecols='A:I')
    data_skripsi = data_skripsi[data_skripsi['Abstrak'].notnull()]
    data_skripsi.reset_index(drop=True, inplace=True)
    print(data_skripsi[['Judul Skripsi', 'Abstrak']])

# Fungsi untuk menampilkan data skripsi
def display_grafik_skripsi():
    # Muat data yang sudah dibersihkan
    data_skripsi = pd.read_csv('cleaned_data_skripsi.csv')

    # Hitung jumlah kemunculan setiap dosen sebagai Dosen Pembimbing 1
    dospem1_count = data_skripsi['Dospem1'].value_counts()

    # Hitung jumlah kemunculan setiap dosen sebagai Dosen Pembimbing 2
    dospem2_count = data_skripsi['Dospem2'].value_counts()

    # Plot grafik batang untuk Dosen Pembimbing 1
    plt.figure(figsize=(10, 6))
    bars1 = dospem1_count.plot(kind='bar', color='lightcoral')
    
    # Judul dan label sumbu untuk grafik Dosen Pembimbing 1
    plt.title('Distribusi Dosen Pembimbing 1', fontsize=14)
    plt.xlabel('Nama Dosen Pembimbing 1', fontsize=12)
    plt.ylabel('Jumlah Skripsi Dibimbing', fontsize=12)

    # Menambahkan keterangan jumlah skripsi di atas setiap batang untuk Dospem 1
    for index, value in enumerate(dospem1_count):
        plt.text(index, value + 0.1, str(value), ha='center', fontsize=10)

    # Tampilkan grafik Dospem 1
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Plot grafik batang untuk Dosen Pembimbing 2
    plt.figure(figsize=(10, 6))
    bars2 = dospem2_count.plot(kind='bar', color='lightblue')

    # Judul dan label sumbu untuk grafik Dosen Pembimbing 2
    plt.title('Distribusi Dosen Pembimbing 2', fontsize=14)
    plt.xlabel('Nama Dosen Pembimbing 2', fontsize=12)
    plt.ylabel('Jumlah Skripsi Dibimbing', fontsize=12)

    # Menambahkan keterangan jumlah skripsi di atas setiap batang untuk Dospem 2
    for index, value in enumerate(dospem2_count):
        plt.text(index, value + 0.1, str(value), ha='center', fontsize=10)

    # Tampilkan grafik Dospem 2
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

def display_wordcloud_dospem():
    # Muat data yang sudah dibersihkan
    data_skripsi = pd.read_csv('cleaned_data_skripsi.csv')
    # Hitung jumlah kemunculan setiap dosen sebagai Dospem1 dan Dospem2
    dospem1_count = data_skripsi['Dospem1'].value_counts()
    dospem2_count = data_skripsi['Dospem2'].value_counts()
    combined_dospem = dospem1_count.add(dospem2_count, fill_value=0)

    # Buat WordCloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(combined_dospem)

    # Plot WordCloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title("Distribusi Dosen Pembimbing (Dospem1 dan Dospem2) - WordCloud", fontsize=16)
    plt.show()

# Fungsi untuk melakukan rekomendasi skripsi hanya berdasarkan abstrak
def recommend_skripsi():
    data_skripsi = pd.read_csv('cleaned_data_skripsi.csv')
    
    # Membuat TF-IDF untuk judul dan abstrak
    tfidf_vectorizer_judul = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, max_df=0.85, sublinear_tf=True)
    tfidf_vectorizer_abstrak = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, max_df=0.85, sublinear_tf=True)
    
    # Membuat matriks TF-IDF
    tfidf_matrix_judul = tfidf_vectorizer_judul.fit_transform(data_skripsi['cleaned_judul'])
    tfidf_matrix_abstrak = tfidf_vectorizer_abstrak.fit_transform(data_skripsi['cleaned_abstrak'])

    # Fungsi untuk menghitung rekomendasi
    def recommendations(keyword, top=10):
        rekomendasi = []
        cleaned_keyword = text_clean(keyword)

        # TF-IDF untuk input keyword
        tfidf_keyword_judul = tfidf_vectorizer_judul.transform([cleaned_keyword])
        tfidf_keyword_abstrak = tfidf_vectorizer_abstrak.transform([cleaned_keyword])
        
        # Menghitung cosine similarity
        scores_judul = cosine_similarity(tfidf_matrix_judul, tfidf_keyword_judul).flatten()
        scores_abstrak = cosine_similarity(tfidf_matrix_abstrak, tfidf_keyword_abstrak).flatten()
        
        # Menjumlahkan skor dari judul dan abstrak
        combined_scores = scores_judul + scores_abstrak
        
        # Mengurutkan berdasarkan skor tertinggi
        sorted_indexes = np.argsort(combined_scores)[::-1]
        
        # Mendapatkan rekomendasi berdasarkan top skor
        for i in sorted_indexes[:top]:
            judul = data_skripsi.iloc[i]['Judul Skripsi']
            link = data_skripsi.iloc[i]['Link Skripsi']
            penulis = data_skripsi.iloc[i]['Penulis']
            abstrak = data_skripsi.iloc[i]['Abstrak']
            score = combined_scores[i]
            rekomendasi.append((judul, link, penulis, abstrak, score))
        
        return rekomendasi

    # Input keyword dari user
    keyword = input("Masukkan keyword: ")
    jumlah_rekomendasi = int(input("Masukkan jumlah rekomendasi: "))
    
    # Mendapatkan rekomendasi
    hasil_rekomendasi = recommendations(keyword, jumlah_rekomendasi)
    
    # Menampilkan hasil
    print(f"Hasil rekomendasi skripsi yang mungkin Anda sukai berdasarkan '{keyword}':")
    for index, (judul, link, penulis, abstrak, score) in enumerate(hasil_rekomendasi, 1):
        print(f"\nRekomendasi ke-{index}:")
        print(f"Judul: {judul}")
        print(f"Link: {link}")
        print(f"Penulis: {penulis}")
        print(f"Abstrak: {abstrak}")
        print(f"Score: {score:.4f}")

# Menu
def main():
    # Jalankan cleaning abstract pertama kali saat sistem dimulai
    cleaning_judulabstract()

    while True:
        print("========== MENU ==========")
        print("1. Data Skripsi")
        print("2. Grafik Skripsi di Sistem Informasi")
        print("3. Rekomendasi Skripsi Berdasarkan Abstrak")
        print("0. Keluar")
        choice = input("Pilih menu: ")

        if choice == "1":
            print("\n----- Data Skripsi -----")
            display_data_skripsi()
            print("-----------------------\n")
        elif choice == "2":
            print("\n----- Grafik Skripsi di Sistem Informasi -----")
            display_grafik_skripsi()
            print("-----------------------------------\n")
        elif choice == "3":
            print("\n----- Rekomendasi Skripsi Berdasarkan Abstrak -----")
            recommend_skripsi()
            print("-----------------------------\n")
        elif choice == "0":
            print("Terima kasih! Program telah berakhir.")
            break
        else:
            print("Pilihan tidak valid. Silakan pilih menu yang tersedia")


# Jalankan program
if __name__ == "__main__":
    main()

Data baru ditemukan, melakukan proses NLP Cleaning
Proses Cleaning pada data telah selesai, data telah di simpan
========== MENU ==========
1. Data Skripsi
2. Grafik Skripsi di Sistem Informasi
3. Rekomendasi Skripsi Berdasarkan Abstrak
0. Keluar
Terima kasih! Program telah berakhir.
